In the same directory, there is `talents.txt` file. That is PDF content provided by Gallup(tm) please note that they are owner of this content.

Countries taken from here: https://gist.github.com/pamelafox/986163 - I had to do some changes, that is why I have a copy.

To run this script pandas, folium, xlrd, xlwt modules are needed.

Have a fun!

Take `talents.txt`, and creates `talents2.xls` with geo coordinates. 

In [1]:
from helper import convert_to_excel, legend

#convert_to_excel('talents.txt', 'talents2.xls')

Read dataframe from `talents2.xls`

In [2]:
import pandas as pd

df = pd.read_excel('talents2.xls')

remove females, males, all entries (the ones without coordinates)

In [3]:
df = df.dropna(subset=['lon'])
df = df.dropna(subset=['lat'])

Print out map with Learners!

In [4]:
import folium
from folium import plugins

sections = [
    {'color': "#007849", 'fill_color':"#015627",
     'description': "Coverage",
     'start_angle':0, 'stop_angle':120},
    {'color': "#8BB0EC", 'fill_color':"#6990BA",
     'description': "Respondents",
     'start_angle':121, 'stop_angle':240},
    {'color': "#FF3C3C", 'fill_color':"#BB0909",
     'description': "Population",
     'start_angle':241, 'stop_angle':360}
]

def put_piechart(folium_map, location, row):
    popup_text = f"country: {row['country']}<br> respondants: {row['respondents']}"
    entries = []
    if row['respondents'] and row['population']:
        cover =  10000000.0 * float(row['respondents']) / float(row['population'])
    else:
        cover = 1
    e = sections[0].copy()
    e['radius'] = cover
    entries.append(e)
    if row['respondents']:
        respondents = float(row['respondents']) / 10.0
    else:
        respondents = 1
    e = sections[1].copy()
    e['radius'] = respondents
    entries.append(e)
    if row['population']:
        population = float(row['population']) / 1000.0
    else:
        population = 1
    e = sections[2].copy()
    e['radius'] = population
    entries.append(e)

    for e in entries:
        plugins.SemiCircle(location=(row['lat'], row['lon']), 
                           radius=e['radius'], 
                           start_angle=e['start_angle'], stop_angle=e['stop_angle'],
                           color=e['color'], fill_color=e['fill_color'], fill=True, fill_opacity=0.7,
                           popup=popup_text, tooltip=f"{row['country']}"
                          ).add_to(folium_map)


folium_map = folium.Map(location=(32.2297700, 21.0117800), zoom_start=2)

for index, row in df.iterrows():
    put_piechart(folium_map, location=(row["lat"], row["lon"]), row=row)
    
legend(folium_map, sections)

Sample output stored in html - github is not supporting folium....

In [5]:
#folium_map.save("sample_demography.html")

Print out output on screen.

In [6]:
folium_map